In [8]:
import numpy as np
import pandas as pd

In [9]:
#https://www.ers.usda.gov/data-products/food-access-research-atlas
df_food = pd.read_csv("raw_data/usda/2019 Food Access Research Atlas Data/Food Access Research Atlas.csv")

geog = pd.read_csv('derived_data/geography.csv')

In [10]:
cols = [
    "CensusTract",
    "lahunv1share",
    "lahunv1",
    "lahunv10share",
    "lahunv10",
    "lasnap1share",
    "lasnap1",
    "lasnap10share",
    "lasnap10",
    "TractHUNV",
    "TractSNAP",
]

df_food = df_food[cols].rename(columns={"CensusTract": "TRACT"})

In [11]:
food_data_tract = df_food[[
    "TRACT",
    "lahunv1share",
    "lahunv10share",
    "lasnap1share",
    "lasnap10share",
]].drop_duplicates()

In [12]:
df_food = geog.merge(df_food, on="TRACT")

df_food["lahunv1"] *= df_food["TZ_RATIO"]
df_food["lahunv10"] *= df_food["TZ_RATIO"]
df_food["lasnap1"] *= df_food["TZ_RATIO"]
df_food["lasnap10"] *= df_food["TZ_RATIO"]
df_food["TractHUNV"] *= df_food["TZ_RATIO"]
df_food["TractSNAP"] *= df_food["TZ_RATIO"]

In [13]:
gb = df_food.groupby("ZIP")
data = []
for zid, sub_df in gb:
    num, den = sub_df[["lahunv1", "TractHUNV"]].dropna().sum(axis=0).values
    lahunv1 = num / den if den > 0 else np.nan
    num, den = sub_df[["lahunv10", "TractHUNV"]].dropna().sum(axis=0).values
    lahunv10 = num / den if den > 0 else np.nan
    num, den = sub_df[["lasnap1", "TractSNAP"]].dropna().sum(axis=0).values
    lasnap1 = num / den if den > 0 else np.nan
    num, den = sub_df[["lasnap10", "TractSNAP"]].dropna().sum(axis=0).values
    lasnap10 = num / den if den > 0 else np.nan
    data.append([zid, lahunv1, lahunv10, lasnap1, lasnap10])

food_data_zcta = pd.DataFrame(data, columns=[    
    "ZIP",
    "lahunv1share",
    "lahunv10share",
    "lasnap1share",
    "lasnap10share",
])

In [14]:
gb = df_food.groupby("COUNTY")
data = []
for zid, sub_df in gb:
    num, den = sub_df[["lahunv1", "TractHUNV"]].dropna().sum(axis=0).values
    lahunv1 = num / den if den > 0 else np.nan
    num, den = sub_df[["lahunv10", "TractHUNV"]].dropna().sum(axis=0).values
    lahunv10 = num / den if den > 0 else np.nan
    num, den = sub_df[["lasnap1", "TractSNAP"]].dropna().sum(axis=0).values
    lasnap1 = num / den if den > 0 else np.nan
    num, den = sub_df[["lasnap10", "TractSNAP"]].dropna().sum(axis=0).values
    lasnap10 = num / den if den > 0 else np.nan
    data.append([zid, lahunv1, lahunv10, lasnap1, lasnap10])

food_data_county = pd.DataFrame(data, columns=[    
    "COUNTY",
    "lahunv1share",
    "lahunv10share",
    "lasnap1share",
    "lasnap10share",
])

In [15]:
food_data_county.to_csv("derived_data/county/usda_food_insecure.csv", index=False)
food_data_zcta.to_csv("derived_data/zcta/usda_food_insecure.csv", index=False)
food_data_tract.to_csv("derived_data/tract/usda_food_insecure.csv", index=False)